## Imports

In [165]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.special import boxcox1p

# ml related imports
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn import metrics 
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from catboost import CatBoostRegressor

# silence settingWithCopyWarning
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [126]:
# get the data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [138]:
# test shape
train.shape, test.shape

((1460, 81), (1459, 80))

## Preprocessing

In [139]:
train_pre = train.copy()
test_pre = test.copy()

In [140]:
# combine data
all_data = pd.concat([train_pre, test_pre], ignore_index=True)

# a lot of the missing values are just encodings for the instance that a specific feaure isn't available
# list of features with worng encodign for NA
feature_NA = ['Alley', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 
              'MiscFeature', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']

# assign NA to None to indicate the lack of a certain feature
all_data[feature_NA] = all_data[feature_NA].fillna('None')

# imute missing categorical features mostly with the mode
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data['Utilities'] = all_data['Utilities'].fillna(all_data['Utilities'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['MasVnrType'] = all_data['MasVnrType'].fillna('None')
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Functional'] = all_data['Functional'].fillna(all_data['Functional'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

# imput missing numerical features (most numercial had only 1-2 missing values in that case I just imputet 0)
all_data['MasVnrArea'] = all_data['MasVnrArea'].fillna(0)
all_data['BsmtFinSF1'] = all_data['BsmtFinSF1'].fillna(0)
all_data['BsmtFinSF2'] = all_data['BsmtFinSF2'].fillna(0)
all_data['BsmtUnfSF'] = all_data['BsmtUnfSF'].fillna(0)
all_data['TotalBsmtSF'] = all_data['TotalBsmtSF'].fillna(0)
all_data['BsmtFullBath'] = all_data['BsmtFullBath'].fillna(0)
all_data['BsmtHalfBath'] = all_data['BsmtHalfBath'].fillna(0)
all_data['GarageCars'] = all_data['GarageCars'].fillna(0)
all_data['GarageArea'] = all_data['GarageArea'].fillna(0)
all_data['GarageYrBlt'] = all_data['GarageYrBlt'].fillna(0)
# Neighorhood should impact the size of of street connected to the property
# code from https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

# transform features into different types
# MSSubClasee
all_data['MSSubClass'] = all_data['MSSubClass'].astype('str')
# month sold
MoSold_dict = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
all_data['MoSold'] = all_data['MoSold'].map(MoSold_dict)

# convert categorical to ordinal features 
ord_feats = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
             'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
map_dict_ord = {'None': 0, 'Po': 1, 'Fa': 2, 'TA':3, 'Gd':4, 'Ex': 5, 'No': 1, 'Mn': 2, 'Av': 3, 'Unf': 1, 
                'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
for ord_ in ord_feats:
    all_data[ord_] = all_data[ord_].map(map_dict_ord)

# convert categorical to binary
bin_feats = ['Street', 'Utilities', 'CentralAir']
map_dict_bin = {'Pave': 0, 'Grvl': 1, 'AllPub': 0, 'NoSeWa': 1, 'Y': 1, 'N': 0}
for bin_ in bin_feats:
    all_data[bin_] = all_data[bin_].map(map_dict_bin)

In [141]:
# check missing values
# should only return SalePrice with 1459 missing values
isnull = all_data.isnull().sum()
isnull[isnull > 0]

SalePrice    1459
dtype: int64

In [142]:
id_ = all_data['Id']
all_data.drop(columns='Id', inplace=True)
target = all_data['SalePrice']
all_data.drop(columns='SalePrice', inplace=True)
num_feats = all_data.select_dtypes(exclude='object')
cat_feats = all_data.select_dtypes('object')

In [143]:
id_.shape, target.shape, num_feats.shape, cat_feats.shape, all_data.shape

((2919,), (2919,), (2919, 50), (2919, 29), (2919, 79))

### Feature Engineering
- log transform target
- add all square feet variables to find the total square feet of the house
- transform highly skewed features with box cox
    - we need to use boxcox1p, to account for the zeros in the dataset
    - I will transform every numerical feature with a skeweness higher than 1 and below -1

In [144]:
target = np.log(target)

In [145]:
num_feats['TotalSF'] = num_feats['TotalBsmtSF'] + num_feats['1stFlrSF'] + num_feats['2ndFlrSF']

#### Log transform

In [146]:
# get absloute skewness values for each num feature
skew_feats_o = abs(num_feats.skew())
# transform those with a skewness higher than 0.75
skew_feats = skew_feats_o[skew_feats_o > 0.75]
skew_list = list(skew_feats.index)
# defina lambda, lambda of 0 is equvalent of log transformation
lam = 0.15
for feat in skew_list:
    num_feats[feat] = boxcox1p(num_feats[feat], lam)

### Creating dummies for categorical variables

In [147]:
# dummy varibales
dummy_feats = list(cat_feats)
df_dummy = pd.get_dummies(all_data[dummy_feats], drop_first=True)

#### Combine datasets

In [148]:
final_df = pd.concat([id_, target, num_feats, df_dummy], axis=1)

### Feature Selection 

In [149]:
# drop columns with no predictive value
final_df.drop(columns=['Utilities'], inplace=True)

### Split data in train and test

In [150]:
# split all_data in train and test to perform more preprocessing and feature engineering speratly (prevent data leakage)
train = final_df.loc[:train.shape[0]-1]
test = final_df.loc[train.shape[0]:]

In [151]:
# rest index so that the index start at 0
test.reset_index(drop=True, inplace=True)

In [152]:
train.shape, test.shape

((1460, 242), (1459, 242))

### Drop outliers

In [155]:
# remove outliers 
train = train.drop(train[(train['GrLivArea']>boxcox1p(4000, lam)) & (train['SalePrice']<np.log(300000))].index)

In [156]:
train.shape, test.shape

((1458, 242), (1459, 242))

### save ID

In [157]:
# save id 
Id = test['Id']
# drop id
test.drop(columns='Id', inplace=True)
train.drop(columns='Id', inplace=True)

In [158]:
train.shape, test.shape

((1458, 241), (1459, 241))

## Modeling

In [159]:
# use train set to make x_train and y_train
x_train = train.drop(columns=['SalePrice'])
y_train = train['SalePrice']
test.drop(columns='SalePrice', inplace=True)

In [160]:
x_train.shape, y_train.shape, test.shape

((1458, 240), (1458,), (1459, 240))

In [161]:
# check if x_train and test_pre are identical
list(set(x_train) - set(test)), list(set(test) - set(x_train))

([], [])

### Cross Validation

In [162]:
# code: https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmsle= np.sqrt(-cross_val_score(model, x_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmsle)

In [168]:
# XGBoost
model_xgb = XGBRegressor()
# LightGBM
model_lgb = LGBMRegressor()
# CatBoost
model_cat = CatBoostRegressor(verbose=False)
# Lasso Regression
model_lasso = Lasso(random_state=42)
# Ridge Regression
model_ridge = Ridge(random_state=42)
# ElasticNet
model_elnet = ElasticNet(random_state=42)

# list of models for cross validation
models_list = [model_xgb, model_lgb, model_cat, model_lasso, model_ridge, model_elnet]
model_names = ['model_xgb', 'model_lgb', 'model_cat', 'model_lasso', 'model_ridge', 'model_elnet']

for model, name in zip(models_list, model_names):
    print(name + ' rmsle score:')
    print(np.mean(rmsle_cv(model)))
    print('#'*30)

model_xgb rmsle score:
0.13334509338994802
##############################
model_lgb rmsle score:
0.12704825791903968
##############################
model_cat rmsle score:
0.11537198302846566
##############################
model_lasso rmsle score:
0.2665729076443017
##############################
model_ridge rmsle score:
0.11826202886295231
##############################
model_elnet rmsle score:
0.2627933003105244
##############################


It is obvious that lasso and elasticnet perfomes suffers a lot from using box cox transformation. This is due to the different sacels of the numeric data, with some features tranformed and others in their original scale.

### Hyperparameter tuning
> from looking at different Kaggle kernels there is a lot of predictive power in hyperparameter tuning for Lasso and Elastic Net Regression

#### Lasso Regression
- alpha: constant that is multiplied with the L1 penalty regularizer

In [174]:
estimator = Lasso(random_state=42)
params = {'alpha': [1, 0.1, 0.01, 0.001, 0.00001]}

gsc = GridSearchCV(estimator=estimator, param_grid=params, 
                   scoring='neg_mean_squared_error', 
                   verbose=1,
                   n_jobs=-2)

grid_result = gsc.fit(x_train, y_train)
grid_result.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  25 out of  25 | elapsed:    3.5s finished


{'alpha': 0.001}

In [181]:
np.mean(rmsle_cv(Lasso(alpha=0.001, random_state=42)))

0.11570958142263246

> this is a huge improvement for the lasso regrssion from 0.26 to 0.11

#### Elastic Net Regression
- alpha: constant that multiplies with the penalty terms (L1 and L2)
- l1_ration: mixing parameter for L1 and L2 penalties

In [182]:
estimator = ElasticNet(random_state=42)
params = {'alpha': [1, 0.1, 0.01, 0.001, 0.00001],
          'l1_ratio': [0.8, 0.5, 0.2]}

gsc = GridSearchCV(estimator=estimator, param_grid=params, 
                   scoring='neg_mean_squared_error', 
                   verbose=1,
                   n_jobs=-2)

grid_result = gsc.fit(x_train, y_train)
grid_result.best_params_

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  75 out of  75 | elapsed:    6.6s finished


{'alpha': 0.001, 'l1_ratio': 0.2}

In [184]:
np.mean(rmsle_cv(ElasticNet(alpha=0.001, l1_ratio=0.2, random_state=42)))

0.11424849584553462

> also an huge improvement from 0.26 to 0.11

### Blended Model

In [186]:
# define models
model_ridge = Ridge(random_state=42)
model_cat = CatBoostRegressor(verbose=False)
model_lasso = Lasso(alpha=0.001, random_state=42)
model_elnet = ElasticNet(alpha=0.001, l1_ratio=0.2, random_state=42)

# train
#model_xgb.fit(x_train, y_train)
#model_lgb.fit(x_train, y_train)
model_ridge.fit(x_train, y_train)
model_cat.fit(x_train, y_train)
model_lasso.fit(x_train, y_train)
model_elnet.fit(x_train, y_train)

# predict
#xgb_preds = model_xgb.predict(test)
#lgb_preds = model_lgb.predict(test)
ridge_preds = model_ridge.predict(test)
cat_preds = model_cat.predict(test)
lasso_preds = model_lasso.predict(test)
elnet_preds = model_elnet.predict(test)

# calculate
preds = (ridge_preds + cat_preds + lasso_preds + elnet_preds)/4

# create DataFrame for submission
submission = pd.DataFrame()
submission['Id'] = Id
# transform log of SalePrice
submission['SalePrice'] = np.exp(preds)
# save DataFrame
submission.to_csv('blended_model_ht_1.csv', index=False)
# show submission
submission

,Id,SalePrice
0,1461,120104.758252
1,1462,157547.080419
2,1463,184268.693592
3,1464,198023.681177
4,1465,197433.793237
...,...,...
1454,2915,85888.426262
1455,2916,80874.355782
1456,2917,171166.379830
1457,2918,119164.251244
